In [1]:
!pip install catboost
from catboost import CatBoostClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 11.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score, roc_curve, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbpipeline
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Provide Path To Read Data

In [4]:

IDVs = pd.read_csv('/content/drive/MyDrive/dd_data/train_values.csv')
DVs = pd.read_csv('/content/drive/MyDrive/dd_data/train_labels.csv')

Merging ID and Damage Grade

In [5]:
# Concat both IDVs and DVs for EDA
df = IDVs.merge(DVs, how="left", on="building_id")
print(df.shape)
df.set_index('building_id', inplace=True)
df.head()

(260601, 40)


,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   geo_level_1_id                          260601 non-null  int64 
 1   geo_level_2_id                          260601 non-null  int64 
 2   geo_level_3_id                          260601 non-null  int64 
 3   count_floors_pre_eq                     260601 non-null  int64 
 4   age                                     260601 non-null  int64 
 5   area_percentage                         260601 non-null  int64 
 6   height_percentage                       260601 non-null  int64 
 7   land_surface_condition                  260601 non-null  object
 8   foundation_type                         260601 non-null  object
 9   roof_type                               260601 non-null  object
 10  ground_floor_type                       260601 non-

In [7]:
df.describe(include='all')

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
count,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601,260601,260601,...,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000,260601.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,r,n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216757,219196,182842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,13.900353,701.074685,6257.876148,2.129723,26.535029,8.018051,5.434365,NaN,NaN,NaN,...,0.033626,0.008101,0.000940,0.000361,0.001071,0.000188,0.000146,0.000088,0.005119,2.238272
std,8.033617,412.710734,3646.369645,0.727665,73.565937,4.392231,1.918418,NaN,NaN,NaN,...,0.180265,0.089638,0.030647,0.018989,0.032703,0.013711,0.012075,0.009394,0.071364,0.611814
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,7.000000,350.000000,3073.000000,2.000000,10.000000,5.000000,4.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,12.000000,702.000000,6270.000000,2.000000,15.000000,7.000000,5.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,21.000000,1050.000000,9412.000000,2.000000,30.000000,9.000000,6.000000,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


In [8]:
## Outlier present in house age column


In [9]:
df.corr()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
geo_level_1_id,1.000000,-0.061405,0.002718,-0.089364,-0.003908,0.071158,-0.063474,-0.018245,-0.152038,0.007677,...,0.001911,0.023523,0.003700,0.002977,0.002655,-0.002303,0.001060,0.000523,-0.017992,-0.072347
geo_level_2_id,-0.061405,1.000000,0.000921,0.047730,0.012594,-0.049443,0.035516,0.015833,0.076491,-0.008743,...,-0.008439,-0.030704,-0.004840,-0.004856,0.000687,-0.000757,-0.000152,0.001926,-0.013068,0.043161
geo_level_3_id,0.002718,0.000921,1.000000,-0.021646,-0.006385,-0.005643,-0.024507,-0.015732,0.026294,0.019579,...,-0.002001,-0.007356,-0.007058,-0.004373,-0.000862,-0.002632,-0.000943,0.000269,-0.002463,0.007932
count_floors_pre_eq,-0.089364,0.047730,-0.021646,1.000000,0.086668,0.101071,0.772734,0.174852,-0.027116,-0.041058,...,0.077120,0.035425,0.016384,0.008833,-0.002611,0.006786,0.009639,0.003939,-0.002073,0.122308
age,-0.003908,0.012594,-0.006385,0.086668,1.000000,-0.004323,0.061074,0.068032,0.001321,0.009129,...,-0.010021,0.001193,-0.004189,-0.003514,-0.003658,-0.002169,-0.001764,-0.001195,-0.004534,0.029273
area_percentage,0.071158,-0.049443,-0.005643,0.101071,-0.004323,1.000000,0.196645,0.026287,-0.225541,-0.000473,...,0.159885,0.105983,0.052212,0.050164,0.019421,0.015109,0.015290,0.004983,0.013111,-0.125221
height_percentage,-0.063474,0.035516,-0.024507,0.772734,0.061074,0.196645,1.000000,0.149725,-0.106573,-0.020260,...,0.123551,0.068909,0.031366,0.020032,0.001946,0.011192,0.014660,0.004048,0.005397,0.048130
has_superstructure_adobe_mud,-0.018245,0.015833,-0.015732,0.174852,0.068032,0.026287,0.149725,1.000000,-0.306861,0.006962,...,-0.012642,-0.003935,-0.004281,-0.002369,0.001762,-0.003292,-0.002648,-0.001493,-0.010074,0.055314
has_superstructure_mud_mortar_stone,-0.152038,0.076491,0.026294,-0.027116,0.001321,-0.225541,-0.106573,-0.306861,1.000000,-0.034046,...,-0.159532,-0.117948,-0.036064,-0.023070,-0.025507,-0.008763,-0.011904,-0.003380,0.005628,0.291325
has_superstructure_stone_flag,0.007677,-0.008743,0.019579,-0.041058,0.009129,-0.000473,-0.020260,0.006962,-0.034046,1.000000,...,-0.008868,-0.011632,-0.000971,-0.001362,-0.004240,-0.002586,0.002959,0.000472,0.002127,0.066039


In [10]:
df.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [11]:
cols = df.columns
num_cols = df._get_numeric_data().columns
categorical_cols = list(set(cols) - set(num_cols))
print(categorical_cols)

['ground_floor_type', 'foundation_type', 'plan_configuration', 'roof_type', 'legal_ownership_status', 'position', 'land_surface_condition', 'other_floor_type']


In [12]:
df.shape

(260601, 39)

In [13]:
df.age.unique()

array([ 30,  10,  25,   0,  15,  20,  45,  55,   5,  40,  80,  60,  35,
        70,  50,  65, 100,  75,  85, 190, 995, 105,  90, 120,  95, 110,
       115, 150, 200, 130, 125, 140, 155, 160, 175, 135, 145, 195, 180,
       165, 170, 185])

Handling Outliers

In [14]:
def cap_data(df):
    for col in df.columns:
        print("capping the ",col)
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.01,0.99]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

In [15]:
df.dtypes.unique()

array([dtype('int64'), dtype('O')], dtype=object)

In [16]:
Xcat = df.drop(["damage_grade"], axis=1)
Ycat = df["damage_grade"]


In [17]:
Xcat

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Xcat = cap_data(Xcat)

capping the  geo_level_1_id
capping the  geo_level_2_id
capping the  geo_level_3_id
capping the  count_floors_pre_eq
capping the  age
capping the  area_percentage
capping the  height_percentage
capping the  land_surface_condition
capping the  foundation_type
capping the  roof_type
capping the  ground_floor_type
capping the  other_floor_type
capping the  position
capping the  plan_configuration
capping the  has_superstructure_adobe_mud
capping the  has_superstructure_mud_mortar_stone
capping the  has_superstructure_stone_flag
capping the  has_superstructure_cement_mortar_stone
capping the  has_superstructure_mud_mortar_brick
capping the  has_superstructure_cement_mortar_brick
capping the  has_superstructure_timber
capping the  has_superstructure_bamboo
capping the  has_superstructure_rc_non_engineered
capping the  has_superstructure_rc_engineered
capping the  has_superstructure_other
capping the  legal_ownership_status
capping the  count_families
capping the  has_secondary_use
capping t

In [19]:
Xcat.age.unique()

array([ 30,  10,  25,   0,  15,  20,  45,  55,   5,  40,  80,  60,  35,
        70,  50,  65, 100,  75,  85,  90,  95])

Doing Train Test Split

In [27]:

categorical_features_indices = np.where(Xcat.dtypes != np.int)[0]
Xcat_train, Xcat_test, Ycat_train, Ycat_test = train_test_split(Xcat, Ycat, stratify=Ycat, test_size=0.2, random_state=42)

In [28]:
categorical_features_indices 


array([ 7,  8,  9, 10, 11, 12, 13, 25])

In [29]:
#Model Training

In [30]:
model = CatBoostClassifier(verbose=True,random_state=42, loss_function='MultiClass')

model.fit(Xcat_train, Ycat_train,cat_features=categorical_features_indices,  eval_set=(Xcat_test, Ycat_test))

In [32]:
y_pred = model.predict(Xcat_test)



In [ ]:
#Model Prediction

In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score


In [22]:
accuracy_score(Ycat_test, y_pred)
f1_score(Ycat_test, y_pred, average='micro')

# USING HYPERPARAMETER OPTIMMIZATION OPTUNA LIBRARY

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def objective(trial):

    X_train, X_test, y_train, y_test = Xcat_train, Xcat_test, Ycat_train, Ycat_test
    param = {
        "objective": trial.suggest_categorical("objective", ["MultiClass"]),
        "learning_rate":trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "random_strength":trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "od_type":trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait":trial.suggest_int("od_wait", 10, 50),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        

    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = CatBoostClassifier(**param)

    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)],cat_features=categorical_features_indices, verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_test)
    pred_labels = np.rint(preds)
    score = f1_score(y_test, preds, average='micro')
    return score




if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-04-19 22:37:52,445] A new study created in memory with name: no-name-49c74304-df29-4830-814a-86aab099a59f
[I 2023-04-19 22:46:10,478] Trial 0 finished with value: 0.5689069664818404 and parameters: {'objective': 'MultiClass', 'learning_rate': 0.004687535519510562, 'colsample_bylevel': 0.012628133263089075, 'depth': 4, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'random_strength': 0.7780065878870488, 'od_type': 'IncToDec', 'od_wait': 22, 'l2_leaf_reg': 0.5150201361853007, 'subsample': 0.8331158681673753}. Best is trial 0 with value: 0.5689069664818404.
[I 2023-04-19 23:01:27,313] Trial 1 finished with value: 0.7423303466932715 and parameters: {'objective': 'MultiClass', 'learning_rate': 0.008947976079742567, 'colsample_bylevel': 0.0949235579051135, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'random_strength': 6.094276295625326, 'od_type': 'IncToDec', 'od_wait': 23, 'l2_leaf_reg': 0.0027519847398423084}. Best is trial 1 with value: 0.74233034669

Number of finished trials: 2
Best trial:
  Value: 0.7423303466932715
  Params: 
    objective: MultiClass
    learning_rate: 0.008947976079742567
    colsample_bylevel: 0.0949235579051135
    depth: 9
    boosting_type: Plain
    bootstrap_type: MVS
    random_strength: 6.094276295625326
    od_type: IncToDec
    od_wait: 23
    l2_leaf_reg: 0.0027519847398423084


Using parameters obtained by Optuna for model training

In [ ]:
modelopt = CatBoostClassifier(verbose=True,random_state=0,
                          objective= "MultiClass",
    learning_rate= 0.008947976079742567,
    colsample_bylevel= 0.0949235579051135,
    depth=9,
    boosting_type='Plain',
    bootstrap_type='MVS',
    random_strength=6.094276295625326,
    od_type='IncToDec',
    od_wait=23,
    l2_leaf_reg= 0.0027519847398423084  
    
    )
modelopt.fit(Xcat_train, Ycat_train,cat_features=categorical_features_indices, eval_set=(Xcat_test, Ycat_test))


0:	learn: 1.0950712	test: 1.0950641	best: 1.0950641 (0)	total: 326ms	remaining: 5m 25s
1:	learn: 1.0913506	test: 1.0913372	best: 1.0913372 (1)	total: 794ms	remaining: 6m 36s
2:	learn: 1.0875763	test: 1.0875611	best: 1.0875611 (2)	total: 1.5s	remaining: 8m 17s
3:	learn: 1.0847482	test: 1.0847329	best: 1.0847329 (3)	total: 1.59s	remaining: 6m 36s
4:	learn: 1.0796568	test: 1.0795805	best: 1.0795805 (4)	total: 3.34s	remaining: 11m 4s
5:	learn: 1.0763337	test: 1.0762541	best: 1.0762541 (5)	total: 3.89s	remaining: 10m 45s
6:	learn: 1.0729516	test: 1.0728744	best: 1.0728744 (6)	total: 5.04s	remaining: 11m 54s
7:	learn: 1.0683814	test: 1.0682493	best: 1.0682493 (7)	total: 5.93s	remaining: 12m 15s
8:	learn: 1.0633328	test: 1.0631216	best: 1.0631216 (8)	total: 7.33s	remaining: 13m 26s
9:	learn: 1.0583587	test: 1.0580759	best: 1.0580759 (9)	total: 8.57s	remaining: 14m 8s
10:	learn: 1.0546890	test: 1.0543507	best: 1.0543507 (10)	total: 8.71s	remaining: 13m 3s
11:	learn: 1.0505345	test: 1.0501468	b

In [32]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
y_pred = modelopt.predict(Xcat_test)


print(classification_report(Ycat_test, y_pred))
print(f1_score(Ycat_test, y_pred, average='micro'))